In [19]:
import numpy as np
import time

σ1 = 0.6039
σ2 = 0.3481  
ρ12 = 0.5456

# Fixed parameters
T = 2.0           # Total maturity
N = 200           # Number of time steps
dt = T/N          # Time step size = 0.01
r = 0.0325        # Risk-free rate
S1_0 = 11.08      
S2_0 = 73.4

In [20]:
def generate_correlated_normals(n, ρ):
    """Generate n pairs of correlated standard normal random variables with correlation coefficient ρ"""
    ε1 = np.random.normal(0, 1, n)
    ε2 = ρ * ε1 + np.sqrt(1 - ρ**2) * np.random.normal(0, 1, n)
    return ε1, ε2

In [21]:
def simulate_one_path_euler():
    # Initialize
    S1, S2 = S1_0, S2_0
    
    # Store prices at key time points
    key_prices_S1 = [S1_0]
    key_prices_S2 = [S2_0]
    
    # Step numbers corresponding to key time points
    key_steps = [50, 100, 200]  # 0.5 years, 1.0 years, 2.0 years
    
    for step in range(1, N + 1):
        # Generate correlated random numbers
        Z1, Z2 = generate_correlated_normals(1, ρ12)
        Z1, Z2 = Z1[0], Z2[0]
        
        # Euler discretization (non-exact)
        S1_prev, S2_prev = S1, S2
        
        # Update stock prices
        S1 = S1_prev + r * S1_prev * dt + σ1 * S1_prev * np.sqrt(dt) * Z1
        S2 = S2_prev + r * S2_prev * dt + σ2 * S2_prev * np.sqrt(dt) * Z2
        
        # Prevent negative prices
        S1 = max(S1, 0)
        S2 = max(S2, 0)
        
        # Record key time points
        if step in key_steps:
            key_prices_S1.append(S1)
            key_prices_S2.append(S2)
    
    return key_prices_S1, key_prices_S2


def simulate_paths_vectorized(num_paths):
    """
    Vectorized version that simulates all paths simultaneously
    Only computes prices at key time points to save computation
    """
    # Key time steps: 0, 50, 100, 200 (corresponding to t=0, 0.5, 1.0, 2.0 years)
    key_steps = [0, 50, 100, 200]
    
    # Initialize: shape (num_paths,)
    S1 = np.full(num_paths, S1_0, dtype=np.float64)
    S2 = np.full(num_paths, S2_0, dtype=np.float64)
    
    # Store prices at key time points: shape (num_paths, 4)
    key_prices_S1 = np.zeros((num_paths, 4))
    key_prices_S2 = np.zeros((num_paths, 4))
    key_prices_S1[:, 0] = S1_0
    key_prices_S2[:, 0] = S2_0
    
    # Pre-generate all random numbers for all steps
    # Shape: (num_paths, N) where N=200
    Z1_all = np.random.normal(0, 1, (num_paths, N))
    Z2_independent = np.random.normal(0, 1, (num_paths, N))
    Z2_all = ρ12 * Z1_all + np.sqrt(1 - ρ12**2) * Z2_independent
    
    # Pre-compute constant factors
    drift_factor = 1 + r * dt
    sqrt_dt = np.sqrt(dt)
    
    # Simulate paths
    step_idx = 1
    for step in range(1, N + 1):
        # Get random numbers for this step: shape (num_paths,)
        Z1 = Z1_all[:, step - 1]
        Z2 = Z2_all[:, step - 1]
        
        # Euler discretization (vectorized)
        S1 = S1 * drift_factor + σ1 * S1 * sqrt_dt * Z1
        S2 = S2 * drift_factor + σ2 * S2 * sqrt_dt * Z2
        
        # Prevent negative prices
        S1 = np.maximum(S1, 0)
        S2 = np.maximum(S2, 0)
        
        # Record key time points
        if step in key_steps[1:]:  # Skip step 0 (already recorded)
            key_prices_S1[:, step_idx] = S1
            key_prices_S2[:, step_idx] = S2
            step_idx += 1
    
    return key_prices_S1, key_prices_S2

In [22]:
def calculate_payoff(prices_S1, prices_S2):
    """Calculate option payoff for a single path"""
    # prices[0]: t=0, prices[1]: t=0.5, prices[2]: t=1.0, prices[3]: t=2.0
    
    B1 = min(prices_S1[1]/prices_S1[0], prices_S2[1]/prices_S2[0])
    B2 = min(prices_S1[2]/prices_S1[0], prices_S2[2]/prices_S2[0])
    B3 = min(prices_S1[3]/prices_S1[0], prices_S2[3]/prices_S2[0])
    
    A = (B1 + B2 + B3) / 3.0
    payoff = max(1.0 - A, 0.0)
    
    return payoff


def calculate_payoffs_vectorized(key_prices_S1, key_prices_S2):
    """Calculate option payoffs for all paths (vectorized)"""
    # key_prices shape: (num_paths, 4)
    # [:, 0]: t=0, [:, 1]: t=0.5, [:, 2]: t=1.0, [:, 3]: t=2.0
    
    # Calculate relative returns for each time point
    # Shape: (num_paths,)
    B1 = np.minimum(key_prices_S1[:, 1] / key_prices_S1[:, 0], 
                     key_prices_S2[:, 1] / key_prices_S2[:, 0])
    B2 = np.minimum(key_prices_S1[:, 2] / key_prices_S1[:, 0], 
                     key_prices_S2[:, 2] / key_prices_S2[:, 0])
    B3 = np.minimum(key_prices_S1[:, 3] / key_prices_S1[:, 0], 
                     key_prices_S2[:, 3] / key_prices_S2[:, 0])
    
    # Calculate average
    A = (B1 + B2 + B3) / 3.0
    
    # Calculate payoffs: shape (num_paths,)
    payoffs = np.maximum(1.0 - A, 0.0)
    
    return payoffs

In [23]:
def monte_carlo_euler(num_paths):
    """Original sequential version (kept for comparison)"""
    start_time = time.time()
    
    total_payoff = 0.0
    payoffs = []  # For calculating standard error
    
    for i in range(num_paths):
        prices_S1, prices_S2 = simulate_one_path_euler()
        payoff = calculate_payoff(prices_S1, prices_S2)
        total_payoff += payoff
        payoffs.append(payoff)
    
    # Calculate expected payoff and discount
    expected_payoff = total_payoff / num_paths
    option_price = expected_payoff * np.exp(-r * T)
    
    # Calculate standard error
    payoffs_array = np.array(payoffs)
    standard_error = np.std(payoffs_array) / np.sqrt(num_paths)
    
    end_time = time.time()
    computation_time = end_time - start_time
    
    return option_price, computation_time, standard_error


def monte_carlo_euler_vectorized(num_paths):
    """Optimized vectorized version for fast computation"""
    start_time = time.time()
    
    # Simulate all paths at once
    key_prices_S1, key_prices_S2 = simulate_paths_vectorized(num_paths)
    
    # Calculate all payoffs at once (vectorized)
    payoffs = calculate_payoffs_vectorized(key_prices_S1, key_prices_S2)
    
    # Calculate expected payoff and discount
    expected_payoff = np.mean(payoffs)
    option_price = expected_payoff * np.exp(-r * T)
    
    # Calculate standard error
    standard_error = np.std(payoffs, ddof=0) / np.sqrt(num_paths)
    
    end_time = time.time()
    computation_time = end_time - start_time
    
    return option_price, computation_time, standard_error

In [24]:
# Set random seed for reproducibility
np.random.seed(42)

# Run two different numbers of paths using vectorized version
print("Running optimized vectorized Monte Carlo simulation...")
price_10k, time_10k, se_10k = monte_carlo_euler_vectorized(10000)
price_300k, time_300k, se_300k = monte_carlo_euler_vectorized(300000)

print(f"\nResults:")
print(f"10,000 paths:   Price={price_10k:.4%}, Time={time_10k:.4f}s, Standard Error={se_10k:.6f}")
print(f"300,000 paths:  Price={price_300k:.4%}, Time={time_300k:.4f}s, Standard Error={se_300k:.6f}")

Running optimized vectorized Monte Carlo simulation...

Results:
10,000 paths:   Price=23.2409%, Time=0.1025s, Standard Error=0.002179
300,000 paths:  Price=23.1261%, Time=4.5676s, Standard Error=0.000397
